In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import os

In [4]:
df = pd.read_csv("Resources/exoplanet_data.csv")
df = df.dropna(axis="columns", how="all")
df.dropna()
df.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


In [5]:
X = df[["koi_fpflag_nt","koi_fpflag_ss","koi_fpflag_co","koi_fpflag_ec",
                        "koi_period","koi_period_err1","koi_period_err2",
                        "koi_time0bk","koi_time0bk_err1","koi_time0bk_err2",
                        "koi_impact","koi_impact_err1","koi_impact_err2",
                        "koi_duration","koi_duration_err1","koi_duration_err2",
                        "koi_depth","koi_depth_err1","koi_depth_err2",
                        "koi_prad","koi_prad_err1","koi_prad_err2",
                        "koi_teq","koi_insol","koi_insol_err1","koi_insol_err2",
                        "koi_model_snr","koi_steff","koi_steff_err1","koi_steff_err2",
                        "koi_slogg","koi_slogg_err1","koi_slogg_err2",
                        "koi_srad","koi_srad_err1","koi_srad_err2",
                        "ra","dec","koi_kepmag"]]
y = df["koi_disposition"]

print(X.shape, y.shape)

(6991, 39) (6991,)


In [6]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=42)

In [29]:
from sklearn.preprocessing import MinMaxScaler

X_scale = MinMaxScaler().fit(X_train)

X_train_scaled = X_scale.transform(X_train)
X_test_scaled = X_scale.transform(X_test)

In [30]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()
classifier

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [31]:
classifier.fit(X_train_scaled, y_train)

C:\Users\Mark\anaconda3\envs\PythonData\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [35]:
print(f"Training Data Score: {classifier.score(X_train_scaled, y_train)}")
print(f"Test Data Score: {classifier.score(X_test_scaled, y_test)}")

Training Data Score: 0.8506580202174328
Test Data Score: 0.8415331807780321


In [37]:
predictions = classifier.predict(X_test)

pd.DataFrame({"Prediction": predictions, "Actual": y_test}).reset_index()

,index,Prediction,Actual
0,4982,FALSE POSITIVE,FALSE POSITIVE
1,4866,FALSE POSITIVE,CANDIDATE
2,2934,FALSE POSITIVE,FALSE POSITIVE
3,5007,FALSE POSITIVE,FALSE POSITIVE
4,3869,FALSE POSITIVE,FALSE POSITIVE
...,...,...,...
1743,4006,FALSE POSITIVE,CANDIDATE
1744,6985,FALSE POSITIVE,CANDIDATE
1745,1468,FALSE POSITIVE,CONFIRMED
1746,5138,FALSE POSITIVE,CONFIRMED


In [49]:
from sklearn.model_selection import GridSearchCV
param_grid = {"C": [1,10], 
            'penalty': ["l1", "l2"]}
grid = GridSearchCV(classifier,param_grid, verbose=3)

In [50]:
grid.fit(X_train_scaled, y_train)

Fitting 5 folds for each of 4 candidates, totalling 20 fits
[CV] C=1, penalty=l1 .................................................
[CV] ....................... C=1, penalty=l1, score=nan, total=   0.0s
[CV] C=1, penalty=l1 .................................................
[CV] ....................... C=1, penalty=l1, score=nan, total=   0.0s
[CV] C=1, penalty=l1 .................................................
[CV] ....................... C=1, penalty=l1, score=nan, total=   0.0s
[CV] C=1, penalty=l1 .................................................
[CV] ....................... C=1, penalty=l1, score=nan, total=   0.0s
[CV] C=1, penalty=l1 .................................................
[CV] ....................... C=1, penalty=l1, score=nan, total=   0.0s
[CV] C=1, penalty=l2 .................................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
C:\Users\Mark\anaconda3\envs\PythonData\lib\site-packages\sklearn\model_selection\_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: Solver lbfgs supports only 'l2' or 'none' penalties, got l1 penalty.

  FitFailedWarning)
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
C:\Users\Mark\anaconda3\envs\PythonData\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://sci

[CV] ..................... C=1, penalty=l2, score=0.862, total=   0.4s
[CV] C=1, penalty=l2 .................................................


C:\Users\Mark\anaconda3\envs\PythonData\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


[CV] ..................... C=1, penalty=l2, score=0.834, total=   0.5s
[CV] C=1, penalty=l2 .................................................


...

[CV] .................... C=10, penalty=l2, score=0.853, total=   0.5s
[CV] C=10, penalty=l2 ................................................


C:\Users\Mark\anaconda3\envs\PythonData\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
[Parallel(n_jobs=1)]: Done  20 out of  20 | elapsed:    5.0s finished


[CV] .................... C=10, penalty=l2, score=0.869, total=   0.5s


C:\Users\Mark\anaconda3\envs\PythonData\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


GridSearchCV(cv=None, error_score=nan,
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='auto',
                                          n_jobs=None, penalty='l2',
                                          random_state=None, solver='lbfgs',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='deprecated', n_jobs=None,
             param_grid={'C': [1, 10], 'penalty': ['l1', 'l2']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=3)

In [51]:
print(grid.best_params_)
print(grid.best_score_)

{'C': 10, 'penalty': 'l2'}
0.8668679367481935


In [52]:
import joblib
filename = 'logistic_regression_model.sav'
joblib.dump(classifier, filename)

['logistic_regression_model.sav']